In [1]:
from metaflow import Metaflow, Flow

In [2]:
mf = Metaflow()
mf.flows

[Flow('PubtatorNerFlow'),
 Flow('S3DemoFlow'),
 Flow('BiosentvecFlow'),
 Flow('BioconceptvecFlow')]

In [24]:
flow = Flow("BioconceptvecFlow")
for run in flow:
    print(run)
# run = flow.latest_run

Run('BioconceptvecFlow/1654001290784719')
Run('BioconceptvecFlow/1654000693619793')
Run('BioconceptvecFlow/1654000487866247')
Run('BioconceptvecFlow/1654000032546168')
Run('BioconceptvecFlow/1653999849729928')
Run('BioconceptvecFlow/1653999540252943')
Run('BioconceptvecFlow/1653939173768092')
Run('BioconceptvecFlow/1653938994814781')
Run('BioconceptvecFlow/1653931308615986')
Run('BioconceptvecFlow/1653931269994640')
Run('BioconceptvecFlow/1653931229706502')
Run('BioconceptvecFlow/1653930771180094')
Run('BioconceptvecFlow/1653930623696643')
Run('BioconceptvecFlow/1653929865180821')
Run('BioconceptvecFlow/1653929830862338')
Run('BioconceptvecFlow/1653928846157298')
Run('BioconceptvecFlow/1653927501995615')
Run('BioconceptvecFlow/1653927292553453')
Run('BioconceptvecFlow/1653920996219970')


In [32]:
runs = [_ for _ in flow]
run = runs[-1]

In [33]:
print(run.data)

<MetaflowData: ebi_df, efo_nx, efo_node_df, name>


In [35]:
import json
from typing import List

import numpy as np
import pandas as pd
import sent2vec
from gensim.models import KeyedVectors
from icecream import ic
from metaflow import FlowSpec, step
from scipy.spatial import distance
from pydash import py_
from loguru import logger

from funcs import paths, utils
from funcs.data_processing import pairwise_routine, stage1_processing
from funcs.nlp import nlp


proj_root = utils.find_project_root()
data_root = utils.find_data_root()



ECHO_STEP = 200
MODEL_NAME = "biosentvec-bioconceptvec-skipgram"
OUTPUT_PATH = paths.stage2["output"]
PUBTATOR_DIR = data_root / "output" / "pubtator"
assert PUBTATOR_DIR.exists()

BIOSENTVEC_MODEL_PATH = paths.init["biosentvec_model"]
assert BIOSENTVEC_MODEL_PATH.exists()

BIOCONCEPTVEC_MODEL_PATH = (
    proj_root / "models" / "bioconceptvec" / "bioconceptvec_word2vec_skipgram.bin"
)
assert BIOCONCEPTVEC_MODEL_PATH.exists()

In [36]:
biosentvec_model = sent2vec.Sent2vecModel()
biosentvec_model.load_model(str(BIOSENTVEC_MODEL_PATH))

In [37]:
bioconceptvec_embeddings = KeyedVectors.load_word2vec_format(str(BIOCONCEPTVEC_MODEL_PATH), binary=True)

In [38]:
ebi_df = run.data.ebi_df

In [39]:
ebi_encode_base = nlp.biosentvec_encode_terms(
    text_list=ebi_df["query"].tolist(), biosentvec_model=biosentvec_model
)

2022-05-31 14:59:35.994 | INFO     | funcs.nlp.nlp:_embed:67 - #0 / 1191
2022-05-31 14:59:36.212 | INFO     | funcs.nlp.nlp:_embed:67 - #200 / 1191
2022-05-31 14:59:36.433 | INFO     | funcs.nlp.nlp:_embed:67 - #400 / 1191
2022-05-31 14:59:36.504 | INFO     | funcs.nlp.nlp:_embed:67 - #600 / 1191
2022-05-31 14:59:36.576 | INFO     | funcs.nlp.nlp:_embed:67 - #800 / 1191
2022-05-31 14:59:36.649 | INFO     | funcs.nlp.nlp:_embed:67 - #1000 / 1191


In [42]:
foo = np.array(ebi_encode_base)
foo.shape

(1191, 700)

In [44]:
foo[1, ].shape

(700,)

In [45]:
for idx in range(3):
    print(idx)

0
1
2
